In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from PIL import Image
import numpy as np
import os

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from sklearn.metrics import accuracy_score

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device used: {device.type}')

Device used: cpu


In [2]:
train_size = 0.8                     # In percent
test_size = 1 - train_size           # In percent, calculated dynamically from train_size
num_epochs = 5                      # Number of epochs for the training of the nn
batch_size = 64                      # Size of batches
learning_rate = 0.01               # Learningrate -> subject to change -> learning rate schedule

In [3]:
# Create an empty array to store the image arrays and class
X = []
Y = []

# Define the folder paths containing the images
folder_paths = ['Dataset/Non_Demented/', 'Dataset/Very_Mild_Demented/', 'Dataset/Mild_Demented/', 'Dataset/Moderate_Demented/']
classes = [r'Non demented', r'Very mildly demented', r'mild demented', r'moderate demented']

# Loop over the images to save them in the list
for path in folder_paths:
    c = folder_paths.index(path)
    items = os.listdir(path)
    for picture in items:
        file_path = os.path.join(path, picture)
        # Open the image and convert it to a NumPy array
        img = Image.open(file_path)
        array_representation = np.asarray(img)

        # Append the NumPy array to the list
        X.append(array_representation)
        Y.append(c)

# Convert the list of image arrays to a NumPy arrayF
X = np.array(X)

from sklearn.preprocessing import Normalizer

# Transpose to make each image a row
X = X.reshape(X.shape[0], -1)

# Normalize each row (i.e., each flattened image)
X = Normalizer().fit_transform(X)
X = X.reshape(len(X), 128**2)

# Dynamically calculate the number of classes in dataset
num_classes = len(np.unique(Y))

# Shape of the array
print(np.shape(X))

(6400, 16384)


In [4]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784')
X = mnist.data
y = mnist.target
num_classes = 10

/home/toby_linux/.local/conda/envs/ewha_ml/lib/python3.8/site-packages/sklearn/datasets/_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [5]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# Convert the labels to a pandas Series (if it's not already)
y_series = pd.Series(y)

# Reshape the array to a column vector (required by OneHotEncoder)
y_array = y_series.values.reshape(-1, 1)

# Initialize the OneHotEncoder
encoder = OneHotEncoder(sparse=False, categories='auto')

# Fit and transform the labels
one_hot_encoded = encoder.fit_transform(y_array).astype(int)

# Convert the one-hot encoded array to a Python list
one_hot_encoded_list = one_hot_encoded.tolist()

/home/toby_linux/.local/conda/envs/ewha_ml/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [8]:
Y = one_hot_encoded_list

In [16]:
X = X.to_numpy()

In [17]:
from sklearn.model_selection import train_test_split

# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 1 - train_size, stratify=Y, random_state=42)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 1 - train_size, stratify=Y, random_state=42)
# X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 1 - train_size, stratify=Y_train, random_state=42)

for i in np.unique(Y_train):
    print(f"Elements of {i} in X_train: {np.count_nonzero(Y_train == i)}")
print()
for i in np.unique(Y_test):
    print(f"Elements of {i} in X_test: {np.count_nonzero(Y_test == i)}")
print()
# for i in np.unique(Y_val):
#     print(f"Elements of {i} in X_val: {np.count_nonzero(Y_val == i)}")

Elements of 0 in X_train: 504000
Elements of 1 in X_train: 56000

Elements of 0 in X_test: 126000
Elements of 1 in X_test: 14000



In [18]:
from torch.utils.data import DataLoader, Dataset
# from torchvision import transforms
from torchvision.transforms import v2

# Assuming you have a class named MyDataset for your dataset
class MyDataset(Dataset):
    def __init__(self, X, Y, transform=None):
        self.X = X
        self.Y = Y
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        data_point = self.X[idx]
        label = self.Y[idx]

        if self.transform:
            data_point = self.transform(data_point)

        return data_point, label

# # Define transformations, you can adjust these based on your needs
# transform = transforms.Compose([
#     transforms.ToTensor(),  # Converts to PyTorch tensor
# ])

transformation = v2.Compose([
    v2.ToDtype(torch.float32),
])

# Create an instance of your dataset
dataset_train = MyDataset(X=X_train, Y=Y_train, transform=transformation)
# dataset_val = MyDataset(X=X_val, Y=Y_val, transform=transformation)
dataset_test = MyDataset(X=X_test, Y=Y_test, transform=transformation)

# Define DataLoader for training, validation and test sets
train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
# validation_loader = DataLoader(dataset_val, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(dataset_test, batch_size=batch_size, shuffle=False)

In [19]:
class DenseNet(nn.Module):
    def __init__(self, fc_size, fc_out_features, activation_function=F.relu, dropout_rate_fc=0):
        super(DenseNet, self).__init__()

        self.fc_size = fc_size
        self.fc_out_features = fc_out_features
        self.activation_function = activation_function
        self.dropout_rate_fc = dropout_rate_fc

        # Initialize linear layers as a ModuleList
        self.fc_layers = nn.ModuleList([
            nn.Linear(128**2, fc_size),
            nn.Linear(fc_size, fc_size),
            nn.Linear(fc_size, fc_size),
            nn.Linear(fc_size, fc_size),
            nn.Linear(fc_size, fc_out_features)
        ])

        # Dropout layers
        self.dropout_fc = nn.Dropout(p=dropout_rate_fc)

    def forward(self, x):
        # Dynamically set input size during the first forward pass
        if self.fc_layers[0].weight.size(1) != x.size(1):
            self.fc_layers[0] = nn.Linear(x.size(1), self.fc_size)
            print(self.fc_size)
            self.fc_size = self.fc_size // 4
            
        # Define the forward pass using nn.Sequential
        for i in range(len(self.fc_layers) - 1):
            x = self.activation_function(self.dropout_fc(self.fc_layers[i](x)))

        # Apply softmax during loss computation
        x = self.fc_layers[-1](x)
        return x

In [20]:
X_train_tensor = torch.FloatTensor(X_train)
Y_train_tensor = torch.tensor(Y_train)

In [23]:
import torch.nn.init as init
from torch.optim import lr_scheduler

n_hidden = 4096
criterion = nn.CrossEntropyLoss()

# model = nn.Sequential(nn.Linear(128**2, n_hidden),
#                       nn.ReLU(),
#                       nn.Linear(n_hidden, n_hidden // 4),
#                       nn.ReLU(),
#                       nn.Linear(n_hidden // 4, n_hidden // 16),
#                       nn.ReLU(),
#                       nn.Linear(n_hidden // 16, n_hidden // 256),
#                       nn.ReLU(),
#                       nn.Linear(n_hidden // 256, num_classes),
#                       nn.Softmax(dim=1))

model = nn.Sequential(nn.Linear(784, 256),
                      nn.LeakyReLU(),
                      nn.Linear(256, 128),
                      nn.LeakyReLU(),
                      nn.Linear(128, 64),
                      nn.LeakyReLU(),
                      nn.Linear(64, 32),
                      nn.LeakyReLU(),
                      nn.Linear(32, num_classes),
                      nn.Softmax(dim=1)
                      )

# Initialize the weights using a specific initialization method (e.g., He initialization)
def init_weights(m):
    if isinstance(m, nn.Linear):
        init.kaiming_uniform_(m.weight, mode='fan_in', nonlinearity='relu')
        init.zeros_(m.bias)
model.apply(init_weights)

# print(model)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

losses = []
for epoch in range(100):
    pred_y = model(X_train_tensor)
    loss = criterion(pred_y, Y_train_tensor)
    losses.append(loss.item())

    model.zero_grad()
    loss.backward()

    optimizer.step()
    scheduler.step()
    print(loss)

RuntimeError: Expected floating point type for target with class probabilities, got Long

In [ ]:
test = model(X_train_tensor)
np.unique(torch.argmax(test, dim=1))

array([0, 1, 2, 3, 4, 5, 6, 8, 9])

In [ ]:
# model = DenseNet(fc_size=4096, fc_out_features=num_classes)
# model = DenseNet(fc_size=2024, fc_out_features=num_classes)
# model.train()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

n_total_steps = train_loader.__len__()
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.float()
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Update the learning rate
        scheduler.step()

        if ((i+1) % (int(n_total_steps / 4)) == 0) or (i+1 == n_total_steps):
            print(outputs)
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}, training')

print('Finished Training')
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)


with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(num_classes)]
    n_class_samples = [0 for i in range(num_classes)]
    for images, labels in test_loader:
        images = images.float()
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(num_classes):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')


tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1.,

KeyboardInterrupt: 